
# Hugging Face Transformers

## Introdução ao Hugging Face Transformers

O Hugging Face Transformers é uma biblioteca open-source que oferece uma ampla gama de modelos pré-treinados para tarefas de processamento de linguagem natural (NLP). Ela suporta tanto PyTorch quanto TensorFlow, permitindo flexibilidade na escolha do framework.

## Utilização para SLMs

Para SLMs (Small Language Models), o Hugging Face fornece modelos leves e eficientes, como o DistilBERT e o TinyBERT, que são versões compactas de modelos maiores. Esses modelos são ideais para aplicações com recursos limitados ou que exigem alta velocidade.

## Modelos Pré-treinados Disponíveis

Alguns modelos pré-treinados disponíveis para SLMs incluem:

- **DistilBERT**: Uma versão compacta do BERT com 40% menos parâmetros.
- **TinyBERT**: Um modelo ainda menor com apenas 14.5 milhões de parâmetros.
- **ALBERT**: Um modelo que reduz a memória necessária compartilhando parâmetros.

## Exemplo Prático com Python

### Instalação das Bibliotecas Necessárias

Antes de começar, é necessário instalar as bibliotecas apropriadas para trabalhar com o Hugging Face Transformers.

Este markdown mantém a estrutura organizada e clara, removendo a linha específica de instalação conforme solicitado.

In [1]:
!pip install transformers
!pip install torch  # ou tensorflow, dependendo do framework escolhido


In [2]:
#carregando o modelo pré-treinado
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Escolha do modelo pré-treinado
model_name = "distilbert-base-uncased"

# Carregamento do tokenizer e do modelo
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
# Texto de entrada
text = "Este é um exemplo de texto para classificação."

# Tokenização do texto
inputs = tokenizer(text, return_tensors="pt")

# Obtenção das previsões
outputs = model(**inputs)

# Visualização das previsões
logits = outputs.logits
print(logits)


tensor([[-0.0302,  0.0864]], grad_fn=<AddmmBackward0>)



## Explicação

Este resultado é um tensor PyTorch que representa as saídas brutas (logits) do modelo para a tarefa de classificação. Vamos analisar em detalhes:

1. **Formato do tensor**: 
   - Forma [1, 2]: 1 linha (uma entrada de texto) e 2 colunas (classificação binária).

2. **Valores do tensor**:
   - Logits brutos: [-0.1819, -0.0352]

3. **Significado dos números**:
   - Cada número é a "pontuação" que o modelo atribui a cada classe.
   - O valor mais alto geralmente corresponde à classe prevista.
   - Aqui, -0.0352 > -0.1819, indicando leve inclinação para a segunda classe.

4. **Interpretação**:
   - Para obter probabilidades, aplica-se uma função softmax aos logits.
   - A classe com o logit mais alto é considerada a previsão do modelo.

5. **grad_fn=<AddmmBackward0>**:
   - Parte do sistema de autograd do PyTorch.
   - Mantém informações para cálculo de gradientes durante o treinamento.

## Observações Importantes

- O modelo `distilbert-base-uncased` não foi fine-tuned para uma tarefa específica de classificação.
- Isso explica os logits próximos de zero e a falta de preferência forte por qualquer classe.

## Próximos Passos para Uso Efetivo

1. Fine-tunar o modelo para sua tarefa específica.
2. Aplicar softmax aos logits para obter probabilidades.
3. Mapear as saídas para rótulos de classe específicos.


In [4]:
import torch.nn.functional as F

probabilities = F.softmax(logits, dim=-1)
print(probabilities)

tensor([[0.4709, 0.5291]], grad_fn=<SoftmaxBackward0>)


###  E agora, porque os resultados somados são iguais a 1?


## Explicação

1. **Transformação dos logits em probabilidades:**
   - A função softmax converte os logits brutos em probabilidades, somando 1 e ficando entre 0 e 1.

2. **Interpretação dos valores:**
   - 0.4634 (46.34%): probabilidade da primeira classe
   - 0.5366 (53.66%): probabilidade da segunda classe

3. **Relação com os logits originais:**
   - Logits originais: [-0.1819, -0.0352]
   - O softmax manteve a ordem relativa: o segundo logit era maior, resultando em maior probabilidade

4. **Proximidade das probabilidades:**
   - Probabilidades próximas (46.34% vs 53.66%)
   - Reflete a pequena diferença entre os logits originais

5. **Interpretação do modelo:**
   - Leve inclinação para a segunda classe (53.66%)
   - Confiança não muito alta, dada a proximidade das probabilidades

6. **grad_fn=<SoftmaxBackward0>:**
   - O PyTorch mantém informações para cálculo de gradientes, útil para treinamento

Esta transformação facilita a interpretação, permitindo análise em termos de probabilidades ou porcentagens para cada classe, em vez de valores arbitrários de logits.